In [12]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv('Hindi_TV_Serials.csv')

In [14]:
df.head()

,Name,Ratings,genres,overview,Year,Cast
0,Kyunki Saas Bhi Kabhi Bahu Thi,1.6,"Comedy, Drama, Family",A mother-in-law's struggle to put up with her ...,2000–2008,"Smriti Malhotra-Irani ,Ronit Roy ,Amar Upadhya..."
1,Kahaani Ghar Ghar Kii,2.1,Drama,The show explored the worlds of its protagonis...,2000–2008,"Sakshi Tanwar ,Kiran Karmarkar ,Mita Vashisht ..."
2,Kesar,4.1,Family,"A story about a girl, Kesar, who fights for he...",2002–2007,"Nandini Singh ,Firoz Ali ,Hiten Tejwani ,Yash ..."
3,Kaahin Kissii Roz,6.0,Thriller,Shaina is beautiful and talented. Kunal is sua...,2001–2004,"Yash Tonk ,Sudha Chandran ,Mouli Ganguly ,Shwe..."
4,K. Street Pali Hill,7.4,Thriller,"A story about three friends relation, emotions...",2004–2006,"Ashwini Kalsekar ,Vikram Kapadia ,Jaswir Kaur ..."


In [15]:
df['genres'].value_counts()

genres
Drama                       314
Comedy                       79
Drama, Romance               38
Romance                      32
Reality-TV                   30
                           ... 
Drama, Musical                1
Action, Biography, Drama      1
Romance, Thriller             1
Fantasy, Thriller             1
Comedy, Drama, Mystery        1
Name: count, Length: 95, dtype: int64

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=1, stop_words='english')


tfidf_matrix = tf.fit_transform(df['Name'])
tfidf_matrix

<744x3499 sparse matrix of type '<class 'numpy.float64'>'
	with 4580 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity


cos=cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [21]:
df_new=pd.Series(df.index,index=df['Name']).drop_duplicates()
df_new

Name
Kyunki Saas Bhi Kabhi Bahu Thi           0
Kahaani Ghar Ghar Kii                    1
Kesar                                    2
Kaahin Kissii Roz                        3
K. Street Pali Hill                      4
                                      ... 
Kaamnaa                                739
Dhadkan Zindaggi Kii                   740
Mose Chhal Kiye Jaaye                  741
Jagannath Aur Purvi Ki Dosti Anokhi    742
Shark Tank India                       743
Length: 744, dtype: int64

In [22]:
def recom_movie(Name, cos=cos):
    # Get the index of the movie that matches the title
    idx = df_new[Name]
    
    # Get the pairwsie similarity scores of all movies with that movie
    cos_scores = list(enumerate(cos[idx]))
    
    # Sort the movies based on the similarity scores
    cos_scores = sorted(cos_scores,key=lambda x: x[1],reverse = True) 
    # Get the scores of the 10 most similar movie
    cos_scores = cos_scores[1:11] # will print 10 Recommended Movie 
    
    # Get the movie indices
    Serial_df_new = [i[0] for i in cos_scores]
    
     # Return the top 10 most similar movies
    return df['Name'].iloc[Serial_df_new]

In [25]:

recom_movie('Kahaani Ghar Ghar Kii')

286                  Ghar Jamai
522                 Swaran Ghar
557                Chidiya Ghar
234                Piya Ka Ghar
416              Ghar Ek Mandir
216    Ghar Ki Lakshmi Betiyann
185      Zindagi Mere Ghar Aana
91          Sajan Ghar Jana Hai
740        Dhadkan Zindaggi Kii
375       Bhabi Ji Ghar Par Hai
Name: Name, dtype: object

In [26]:
recom_movie('Koi Laut Ke Aaya Hai')

699                            Bhoot Aaya
102                    Ssshhhh... Koi Hai
603                Jassi Jaissi Koi Nahin
700    Tera Mujhse Hai Pehle Ka Naata Koi
95                            Siya Ke Ram
404                      Shaadi Ke Siyape
151                 P.O.W. Bandi Yuddh Ke
236                            Qubool Hai
464                             Kaun Hai?
636                     Ek Duje Ke Vaaste
Name: Name, dtype: object

In [27]:
import pickle

# Save the TF-IDF vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tf, f)

# Save the cosine similarity matrix
with open('cosine_similarity.pkl', 'wb') as f:
    pickle.dump(cos, f)